# Microorganisms project

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

## Reading the source file (SQL)

The first problem seems to be reading the data file, since I'd really rather not have to load it into a SQL database just to read it into Pandas. But it turns out it's not really a SQL file. Looks like tab-delimited data, not what I expected from a file with a `.sql` extension (i.e., the output of `mysql dump`).

TIL - Juptyer notebooks can [run/output shell commands directly](https://ipython.org/ipython-doc/3/interactive/reference.html#system-shell-access).

In [2]:
!head data/THE.MICROBE.DIRECTORY.sql

data = pd.read_csv('data/THE.MICROBE.DIRECTORY.sql', sep='\t')

1	Viruses	Viruses noname	Viruses noname	Caudovirales	Podoviridae	T7likevirus	Yersinia phage phiYeO3 12	\N	\N	\N	\N	\N	\N	\N	\N	0	Pajunen MI, Kiljunen SJ, Söderholm ME, Skurnik M. Complete genomic sequence of the lytic bacteriophage phiYeO3-12 of Yersinia enterocolitica serotype O:3. J Bacteriol. 2001 Mar;183(6):1928-37.______http://www.ncbi.nlm.nih.gov/genome/?term=txid110457______http://www.phisite.org/main/index.php?nav=phages&nav_sel=features&phage_id=NC_001271%3B39600%3B1%3B1%3B1&ID=null&begin=begin&end=end&strand=both	0	Pajunen MI, Kiljunen SJ, Söderholm ME, Skurnik M. Complete genomic sequence of the lytic bacteriophage phiYeO3-12 of Yersinia enterocolitica serotype O:3. J Bacteriol. 2001 Mar;183(6):1928-37.______http://www.ncbi.nlm.nih.gov/genome/?term=txid110457______http://www.phisite.org/main/index.php?nav=phages&nav_sel=features&phage_id=NC_001271%3B39600%3B1%3B1%3B1&ID=null&begin=begin&end=end&strand=both	0	Pajunen MI, Kiljunen SJ, Söderholm ME, Skurnik M. Complete genomic 

In [3]:
data.head()
data.shape

(8211, 30)

The data dump provided on their website lacks column names, but they also maintain a [github repo](https://github.com/microbe-directory/microbe-directory) with a csv file which _does_ have column names. So let's use that instead.

In [4]:
data2 = pd.read_csv('data/microbe-directory.csv')
data2.head()
data2.shape

(7678, 30)

_le sigh_ 

The data aren't the same in the two sets. The dump on their website has 8211 rows but the github file has 7678 rows.

In [5]:
data.tail()

,1,Viruses,Viruses noname,Viruses noname.1,Caudovirales,Podoviridae,T7likevirus,Yersinia phage phiYeO3 12,\N,\N.1,...,0.2,"Pajunen MI, Kiljunen SJ, Söderholm ME, Skurnik M. Complete genomic sequence of the lytic bacteriophage phiYeO3-12 of Yersinia enterocolitica serotype O:3. J Bacteriol. 2001 Mar;183(6):1928-37.______http://www.ncbi.nlm.nih.gov/genome/?term=txid110457______http://www.phisite.org/main/index.php?nav=phages&nav_sel=features&phage_id=NC_001271%3B39600%3B1%3B1%3B1&ID=null&begin=begin&end=end&strand=both.2",\N.8,\N.9,\N.10,\N.11,0.3,"Pajunen MI, Kiljunen SJ, Söderholm ME, Skurnik M. Complete genomic sequence of the lytic bacteriophage phiYeO3-12 of Yersinia enterocolitica serotype O:3. J Bacteriol. 2001 Mar;183(6):1928-37.______http://www.ncbi.nlm.nih.gov/genome/?term=txid110457______http://www.phisite.org/main/index.php?nav=phages&nav_sel=features&phage_id=NC_001271%3B39600%3B1%3B1%3B1&ID=null&begin=begin&end=end&strand=both.3",0.4,"Pajunen MI, Kiljunen SJ, Söderholm ME, Skurnik M. Complete genomic sequence of the lytic bacteriophage phiYeO3-12 of Yersinia enterocolitica serotype O:3. J Bacteriol. 2001 Mar;183(6):1928-37.______http://www.ncbi.nlm.nih.gov/genome/?term=txid110457______http://www.phisite.org/main/index.php?nav=phages&nav_sel=features&phage_id=NC_001271%3B39600%3B1%3B1%3B1&ID=null&begin=begin&end=end&strand=both.4"
8206,7664,Bacteria,Proteobacteria,Gammaproteobacteria,Chromatiales,Ectothiorhodospiraceae,Thioalkalivibrio,Thioalkalivibrio sp ALJ3,\N,\N,...,1,http://www.bacterio.net/thioalkalivibrio.html,0,http://www.bacterio.net/thioalkalivibrio.html,\N,\N,\N,\N,\N,\N
8207,7665,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Burkholderiaceae,Burkholderia,Burkholderia sp RPE64,\N,\N,...,\N,\N,0,http://www.rxpgonline.com/article1169.html,1,https://microbewiki.kenyon.edu/index.php/Burkh...,0,https://microbewiki.kenyon.edu/index.php/Burkh...,1,https://microbewiki.kenyon.edu/index.php/Burkh...
8208,7666,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Streptomycetaceae,Streptomyces,Streptomyces ghanaensis,\N,\N,...,0,https://img.jgi.doe.gov/cgi-bin/m/main.cgi?sec...,1,https://img.jgi.doe.gov/cgi-bin/m/main.cgi?sec...,0,https://img.jgi.doe.gov/cgi-bin/m/main.cgi?sec...,0,https://img.jgi.doe.gov/cgi-bin/m/main.cgi?sec...,0,https://img.jgi.doe.gov/cgi-bin/m/main.cgi?sec...
8209,7667,Viruses,Viruses noname,Viruses noname,Caudovirales,Siphoviridae,Siphoviridae noname,Pseudomonas phage JBD24,\N,\N,...,0,Bondy-Denomy et al. 2013 (URL: http://ezproxy....,\N,\N,0,Bondy-Denomy et al. 2013 (URL: http://ezproxy....,0,Bondy-Denomy et al. 2013 (URL: http://ezproxy....,0,Bondy-Denomy et al. 2013 (URL: http://ezproxy....
8210,7668,qqq,www,eee,rrr,tttt,yyy,qqqaaaazzz,10.5,https://royjr.com,...,1,https://royjr.com,2,https://royjr.com,0,https://royjr.com,1,https://royjr.com,1,https://royjr.com


In [6]:
data2.tail()

,microbe_id,kingdom,phylum,class,order,family,genus,species,gram_stain,gram_stain_citation,...,optimal_ph,optimal_ph_citation,animal_pathogen,animal_pathogen_citation,spore_forming,spore_forming_citation,pathogenicity,pathogenicity_citation,plant_pathogen,plant_pathogen_citation
7673,7674,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Streptomycetaceae,Streptomyces,Streptomyces lividans,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7674,7675,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Streptomycetaceae,Streptomyces,Streptomyces niveus,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7675,7676,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Streptomycetaceae,Streptomyces,Streptomyces scabrisporus,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7676,7677,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Streptomycetaceae,Streptomyces,Streptomyces scabiei,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7677,7678,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Streptomycetaceae,Streptomyces,Streptomyces purpureus,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Ok. Looks like the SQL dump has some cruft in it, like a row where the kingdom is `qqq` and a bunch of rows where there where line breaks in titles, unescaped quotes, etc. So I'm calling it unclean and using the data from github instead.

## Exploring the data

In [7]:
data = data2
columnDatatypes = data.dtypes

for i, v in columnDatatypes.iteritems():
    print('==================')
    print(f'{i}: {v}\n')
    
    if data[i].dtype.name in ['object', 'category']:
        print(data[i].value_counts())

    else:
        print(f'min: {data[i].min()}')
        print(f'median: {data[i].mean()}')
        print(f'max: {data[i].max()}')
        print(f'std: {data[i].std()}')
    
    print(f'Nulls: {data[i].isnull().sum()}')
    print('------------------')

microbe_id: int64

min: 1
median: 3839.5
max: 7678
std: 2216.5920162868642
Nulls: 0
------------------
kingdom: object

Bacteria     3849
Viruses      3449
Archaea       228
Eukaryota     111
Viroids        41
Name: kingdom, dtype: int64
Nulls: 0
------------------
phylum: object

Viruses noname                 3449
Proteobacteria                 1626
Firmicutes                      833
Actinobacteria                  601
Bacteroidetes                   309
Euryarchaeota                   176
Cyanobacteria                   123
Spirochaetes                     73
Ascomycota                       65
Tenericutes                      56
Viroids noname                   41
Crenarchaeota                    38
Deinococcus Thermus              25
Fusobacteria                     21
Verrucomicrobia                  20
Planctomycetes                   17
Thermotogae                      17
Eukaryota noname                 16
Chloroflexi                      16
Apicomplexa                      1

### Resetting data types
A bunch of these columns are being pulled in as float64, but they're really boolean, all the citations should be strings (objects in Pandas land), and there are some categoricals that aren't marked as such.

In [8]:
## manually set some column types
columnTypes = {
    'gram_stain': 'category',
    'microbiome_location': 'bool',
    'antimicrobial_susceptibility': 'bool',
    'extreme_environment': 'bool',
    'biofilm_forming': 'bool',
    'animal_pathogen': 'bool',
    'spore_forming': 'bool',
    'pathogenicity': 'category',
    'plant_pathogen': 'bool'
}

## Convert citation columns to object/strings.
citationColumns = {key: 'object' for key in data.columns if key.find('_citation') != -1}

convertedData = data.astype({**columnTypes, **citationColumns}, copy=False)


In [9]:
convertedData.dtypes

microbe_id                                  int64
kingdom                                    object
phylum                                     object
class                                      object
order                                      object
family                                     object
genus                                      object
species                                    object
gram_stain                               category
gram_stain_citation                        object
microbiome_location                          bool
microbiome_location_citation               object
antimicrobial_susceptibility                 bool
antimicrobial_susceptibility_citation      object
optimal_temperature                       float64
optimal_temperature_citation               object
extreme_environment                          bool
extreme_environment_citation               object
biofilm_forming                              bool
biofilm_forming_citation                   object


In [10]:
crosstabColumns = [
    'microbiome_location',
    'antimicrobial_susceptibility',
    'extreme_environment',
    'biofilm_forming',
    'animal_pathogen',
    'spore_forming',
    'plant_pathogen'
]

for col in crosstabColumns:
    print(pd.crosstab(convertedData.kingdom, convertedData[col], dropna=False))

microbiome_location  False  True 
kingdom                          
Archaea                206     22
Bacteria              1972   1877
Eukaryota               95     16
Viroids                  0     41
Viruses               1512   1937
antimicrobial_susceptibility  False  True 
kingdom                                   
Archaea                           0    228
Bacteria                        518   3331
Eukaryota                        57     54
Viroids                           0     41
Viruses                         467   2982
extreme_environment  False  True 
kingdom                          
Archaea                 37    191
Bacteria              1985   1864
Eukaryota               89     22
Viroids                 41      0
Viruses               2123   1326
biofilm_forming  False  True 
kingdom                      
Archaea              0    228
Bacteria           219   3630
Eukaryota           24     87
Viroids             41      0
Viruses            948   2501
animal_pathog

In [11]:
# chi-square tests to see if correlation between kingdom and animal pathogenicity

contingencies = pd.crosstab(convertedData.kingdom, convertedData.animal_pathogen)

stats.chi2_contingency(contingencies)

(240.06628004181678,
 8.977883053295579e-51,
 4,
 array([[ 101.34983069,  126.65016931],
        [1710.94516801, 2138.05483199],
        [  49.34136494,   61.65863506],
        [  18.22518885,   22.77481115],
        [1533.13844751, 1915.86155249]]))

In [14]:
archaea = stats.chisquare(contingencies.iloc[0])
bacteria = stats.chisquare(contingencies.iloc[1])
eukaryota = stats.chisquare(contingencies.iloc[2])
viroids = stats.chisquare(contingencies.iloc[3])
viruses = stats.chisquare(contingencies.iloc[4])

In [16]:
print(contingencies)
print(archaea)
print(bacteria)
print(eukaryota)
print(viroids)
print(viruses)





animal_pathogen  False  True 
kingdom                      
Archaea            171     57
Bacteria          1458   2391
Eukaryota           67     44
Viroids             41      0
Viruses           1676   1773
Power_divergenceResult(statistic=57.0, pvalue=4.3581190270320824e-14)
Power_divergenceResult(statistic=226.1597817614965, pvalue=4.1007210586066347e-51)
Power_divergenceResult(statistic=4.7657657657657655, pvalue=0.029031142135425358)
Power_divergenceResult(statistic=41.0, pvalue=1.5222921962563087e-10)
Power_divergenceResult(statistic=2.7280371122064366, pvalue=0.09860040143630405)
